In [ ]:
import os
import numpy as np
import librosa
import soundfile as sf
from pydub import AudioSegment
from IPython.display import Audio
import math
from google.colab import files

# Install dependencies
!apt install ffmpeg
!pip install spleeter librosa soundfile pydub

# Configuration
output_dir = '/content/output/'
stems = 4  # 2, 4, or 5
bass_boost = 6  # dB of bass boost

def bass_line_freq(samples):
    est_mean = np.mean(samples)
    est_std = 3 * np.std(samples) / (math.sqrt(2))
    bass_factor = int(round((est_std - est_mean) * 0.005))
    return max(bass_factor, 60)  # Ensure minimum of 60 Hz

# Upload file
print("Please upload an audio file:")
uploaded = files.upload()

if not uploaded:
    raise ValueError("No file was uploaded.")

input_file = list(uploaded.keys())[0]
input_path = f'/content/{input_file}'

print(f"Processing file: {input_file}")

# Load the audio file using both pydub and librosa
audio = AudioSegment.from_file(input_path)
samples = np.array(audio.get_array_of_samples())
y, sr = librosa.load(input_path)

# Calculate adaptive bass frequency
bass_freq = bass_line_freq(samples)
print(f"Detected bass frequency: {bass_freq} Hz")

# Compute the Short-time Fourier Transform (STFT)
D = librosa.stft(y)

# Convert to magnitude and phase
S, phase = librosa.magphase(D)

# Define the frequency range for bass (detected_freq to 250 Hz)
freq_range = librosa.fft_frequencies(sr=sr)
bass_range = (freq_range >= bass_freq) & (freq_range <= 250)

# Boost the bass frequencies
S[bass_range] *= 10 ** (bass_boost / 20)  # Convert dB to magnitude

# Reconstruct the signal
y_boosted = librosa.istft(S * phase)

# Save the bass-boosted audio
boosted_file = 'bass_boosted_' + input_file
boosted_path = f'/content/{boosted_file}'
sf.write(boosted_path, y_boosted, sr)

print("Original audio:")
display(Audio(input_path))

print("Bass-boosted audio:")
display(Audio(boosted_path))

# Separate audio
print(f"Separating bass-boosted audio into {stems} stems...")
!spleeter separate -p spleeter:{stems}stems -o {output_dir} {boosted_path}

# Get the base name of the input file without extension
base_name = os.path.splitext(boosted_file)[0]

# Display results
print("Separated audio files:")
output_path = os.path.join(output_dir, base_name)
if os.path.exists(output_path):
    for file in os.listdir(output_path):
        print(f"- {file}")
        display(Audio(os.path.join(output_path, file)))

    # Offer to download the separated files
    print("\nDo you want to download the separated files? (yes/no)")
    if input().lower().startswith('y'):
        for file in os.listdir(output_path):
            files.download(os.path.join(output_path, file))
else:
    print("No output files found. Separation may have failed.")